In [1]:
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV
from sklearn.metrics import mean_squared_error
from tqdm.auto import tqdm
from typing import List
import time

sys.path.append("../")

from RGS_test import *
from data_generator import *
from data_plotting import *

In [2]:
# # Set random seed for reproducibility 
# rng = np.random.default_rng(42)

# # Generate synthetic data 
# n_samples = 1000
# n_features = 20
# k_true = 3  # Only first 3 features are relevant

# # Generate features - now in sklearn convention (n_samples, n_features)
# X = rng.standard_normal((n_samples, n_features))

# # Generate target with only first k_true features being relevant
# true_coef = np.zeros(n_features)
# true_coef[:k_true] = [1.0, 0.5, 0.25]  # Decreasing importance
# noise = rng.standard_normal(n_samples) * 0.1
# y = X @ true_coef + noise

# # Initialize model
# k_max = 10
# m_grid = [1, 3, 5, 10, 15, 20]
# cv = KFold(n_splits=5, shuffle=True, random_state=42)
# model = RGSCV(
#     k_max=k_max,
#     m_grid=m_grid,
#     n_replications=1000,
#     random_state=42,
#     cv=cv
# )

# # Fit model
# model.fit(X, y)

# # Print results
# print("True coefficients:", true_coef)
# print("\nSelected k:", model.k_)
# print("Selected m:", model.m_)
# print("\nLearned coefficients:", model.coef_[model.k_])

In [3]:
# import numpy as np
# from sklearn.datasets import make_regression
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import r2_score, mean_squared_error
# import matplotlib.pyplot as plt

# # Generate synthetic dataset
# n_samples = 200
# n_features = 20
# n_informative = 5  # Only 5 features will be truly informative
# random_state = 42

# X, y, coef = make_regression(
#     n_samples=n_samples,
#     n_features=n_features,
#     n_informative=n_informative,
#     noise=0.1,
#     coef=True,
#     random_state=random_state
# )

# # Split the data
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=random_state
# )

# # Initialize and fit RGS
# rgs = RGS(
#     k_max=10,           # Maximum model size to consider
#     m=3,                # Number of random features to sample at each step
#     n_replications=100, # Number of random replications
#     random_state=42
# )

# # Fit the model
# rgs.fit(X_train, y_train)

# # Evaluate performance for different k values
# k_values = range(1, 11)
# train_scores = []
# test_scores = []

# for k in k_values:
#     # Make predictions
#     y_train_pred = rgs.predict(X_train, k=k)
#     y_test_pred = rgs.predict(X_test, k=k)
    
#     # Calculate R² scores
#     train_score = r2_score(y_train, y_train_pred)
#     test_score = r2_score(y_test, y_test_pred)
    
#     train_scores.append(train_score)
#     test_scores.append(test_score)

# # Plot results
# plt.figure(figsize=(10, 6))
# plt.plot(k_values, train_scores, 'b-', label='Training R²')
# plt.plot(k_values, test_scores, 'r--', label='Test R²')
# plt.xlabel('Model Size (k)')
# plt.ylabel('R² Score')
# plt.title('RGS Performance vs Model Size')
# plt.legend()
# plt.grid(True)

# # Print some information about the model
# print("\nModel Information:")
# print(f"Number of features selected at k=5: {len(rgs.feature_sets[5])}")
# print("\nTop 5 most frequent feature sets at k=5:")
# for feature_set, count in rgs.feature_sets[5].most_common(5):
#     print(f"Features {sorted(list(feature_set))}: {count} times")

# # Get predictions for the best performing k
# best_k = np.argmax(test_scores) + 1
# y_pred = rgs.predict(X_test, k=best_k)
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f"\nBest Performance (k={best_k}):")
# print(f"Test MSE: {mse:.4f}")
# print(f"Test R²: {r2:.4f}")

# # Compare with true coefficients
# print("\nTop 5 features by absolute coefficient value at best k:")
# coef_at_k = np.abs(rgs.coef_[best_k])
# top_features = np.argsort(coef_at_k)[-5:][::-1]
# for idx in top_features:
#     print(f"Feature {idx}: {rgs.coef_[best_k][idx]:.4f} (true: {coef[idx]:.4f})")

In [4]:
import numpy as np
from sklearn.datasets import make_regression
from time import time
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import gc

def generate_dataset(n_samples, n_features, n_informative, random_state=42):
    """Generate synthetic dataset with specific characteristics"""
    X, y, coef = make_regression(
        n_samples=n_samples,
        n_features=n_features,
        n_informative=n_informative,
        noise=0.1,
        coef=True,
        random_state=random_state,
        shuffle=True
    )
    return X, y

def compute_mse_for_all_k(model, X, y, k_max):
    """Compute MSE for each value of k"""
    mse_values = []
    for k in range(1, k_max + 1):
        y_pred = model.predict(X, k=k)
        mse = mean_squared_error(y, y_pred)
        mse_values.append(mse)
    return mse_values



def plot_results(results_df, scenarios):
    """Create visualization plots for benchmark results"""
    n_scenarios = len(scenarios)
    
    # Create figure with subplots
    fig = plt.figure(figsize=(15, 5 * (n_scenarios + 1)))
    
    # Plot 1: Speedup comparison
    plt.subplot(n_scenarios + 1, 2, 1)
    plt.bar(results_df['scenario'], results_df['speedup'])
    plt.axhline(y=1, color='r', linestyle='--', alpha=0.5)
    plt.title('Speedup Factor (Original/Optimized)')
    plt.xlabel('Scenario')
    plt.ylabel('Speedup Factor')
    plt.xticks(rotation=45)
    
    # Plot 2: Execution times
    plt.subplot(n_scenarios + 1, 2, 2)
    width = 0.35
    x = np.arange(len(results_df))
    plt.bar(x - width/2, results_df['time_original'], width, label='Original')
    plt.bar(x + width/2, results_df['time_optimized'], width, label='Optimized')
    plt.title('Execution Time Comparison')
    plt.xlabel('Scenario')
    plt.ylabel('Time (seconds)')
    plt.xticks(x, results_df['scenario'], rotation=45)
    plt.legend()
    
    # MSE plots for each scenario
    for i, scenario in enumerate(scenarios):
        # Training MSE
        plt.subplot(n_scenarios + 1, 2, 2*i + 3)
        k_values = range(1, scenario['k_max'] + 1)
        plt.plot(k_values, results_df.iloc[i]['mse_original_train'], 'b-', label='Original (Train)')
        plt.plot(k_values, results_df.iloc[i]['mse_optimized_train'], 'b--', label='Optimized (Train)')
        plt.plot(k_values, results_df.iloc[i]['mse_original_test'], 'r-', label='Original (Test)')
        plt.plot(k_values, results_df.iloc[i]['mse_optimized_test'], 'r--', label='Optimized (Test)')
        plt.title(f'MSE vs k - {scenario["name"]}')
        plt.xlabel('k')
        plt.ylabel('MSE')
        plt.legend()
        plt.grid(True)
    
    plt.tight_layout()
    plt.show()

# # Define benchmark scenarios
# scenarios = [
#     {
#         'name': 'Small Dataset',
#         'n_samples': 100,
#         'n_features': 10,
#         'n_informative': 5,
#         'k_max': 5,
#         'm': 3,
#         'n_replications': 100
#     },
#     {
#         'name': 'Medium Dataset',
#         'n_samples': 1000,
#         'n_features': 50,
#         'n_informative': 10,
#         'k_max': 10,
#         'm': 5,
#         'n_replications': 500
#     },
#     {
#         'name': 'Large Dataset',
#         'n_samples': 5000,
#         'n_features': 100,
#         'n_informative': 20,
#         'k_max': 15,
#         'm': 10,
#         'n_replications': 1000
#     }
# ]


# # Run benchmarks
# results_df = run_benchmark(scenarios)

# # Display results
# print("\nBenchmark Results:")
# pd.set_option('display.max_columns', None)
# print(results_df.to_string(index=False))

# # Plot results
# plot_results(results_df, scenarios)

# # Print validation summary
# print("\nValidation Summary:")
# print("Mean Coefficient Difference:", results_df['coef_diff'].mean())
# print("All Feature Sets Match:", results_df['sets_match'].all())
# print("\nMSE Differences:")
# print("Mean Training MSE Difference:", results_df['mse_diff_train'].mean())
# print("Mean Test MSE Difference:", results_df['mse_diff_test'].mean())

In [5]:
import numpy as np
from sklearn.datasets import make_regression
from time import time
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import gc

class Timer:
    def __init__(self, task_name):
        self.task_name = task_name
        self.elapsed = None

    def __enter__(self):
        gc.collect()
        self.start = time()
        return self

    def __exit__(self, *args):
        self.elapsed = time() - self.start
        print(f"{self.task_name}: {self.elapsed:.2f} seconds")

def compare_feature_sets(original_sets, optimized_sets, k):
    """Compare feature sets and print differences"""
    original_keys = set(original_sets[k].keys())
    optimized_keys = set(optimized_sets[k].keys())
    
    if original_keys != optimized_keys:
        print(f"\nFeature set differences at k={k}:")
        only_in_original = original_keys - optimized_keys
        only_in_optimized = optimized_keys - original_keys
        if only_in_original:
            print(f"Only in original: {sorted([sorted(list(s)) for s in only_in_original])}")
        if only_in_optimized:
            print(f"Only in optimized: {sorted([sorted(list(s)) for s in optimized_keys])}")
        
        common_keys = original_keys & optimized_keys
        for key in common_keys:
            if original_sets[k][key] != optimized_sets[k][key]:
                print(f"Different frequencies for {sorted(list(key))}:")
                print(f"  Original: {original_sets[k][key]}")
                print(f"  Optimized: {optimized_sets[k][key]}")
    return original_keys == optimized_keys

def debug_random_states(scenario):
    """Debug random state behavior"""
    print("\nTesting random state consistency:")
    np.random.seed(42)
    choices1 = np.random.choice(10, size=5, replace=False)
    np.random.seed(42)
    choices2 = np.random.choice(10, size=5, replace=False)
    print(f"Random choices match: {np.array_equal(choices1, choices2)}")
    
    rng1 = np.random.default_rng(42)
    choices3 = rng1.choice(10, size=5, replace=False)
    rng2 = np.random.default_rng(42)
    choices4 = rng2.choice(10, size=5, replace=False)
    print(f"Generator choices match: {np.array_equal(choices3, choices4)}")

def run_benchmark(scenarios):
    """Run benchmarking with detailed debugging"""
    results = []
    
    for scenario in scenarios:
        print(f"\nRunning benchmark for scenario: {scenario['name']}")
        
        # Debug random state behavior
        debug_random_states(scenario)
        
        # Generate dataset
        X, y = generate_dataset(
            n_samples=scenario['n_samples'],
            n_features=scenario['n_features'],
            n_informative=scenario['n_informative']
        )
        
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
        
        # Test original RGS
        rgs_original = RGS(
            k_max=scenario['k_max'],
            m=scenario['m'],
            n_replications=scenario['n_replications'],
            random_state=42
        )
        
        with Timer("Original RGS") as t_original:
            rgs_original.fit(X_train, y_train)
        time_original = t_original.elapsed
        
        # Test optimized RGS
        rgs_optimized = OptimizedRGS(
            k_max=scenario['k_max'],
            m=scenario['m'],
            n_replications=scenario['n_replications'],
            random_state=42
        )
        
        with Timer("Optimized RGS") as t_optimized:
            rgs_optimized.fit(X_train, y_train)
        time_optimized = t_optimized.elapsed
        
        # Detailed feature set comparison
        sets_match = True
        for k in range(scenario['k_max'] + 1):
            if not compare_feature_sets(rgs_original.feature_sets, rgs_optimized.feature_sets, k):
                sets_match = False
                print(f"\nCoefficients at k={k}:")
                print("Original:", rgs_original.coef_[k][:5])  # First 5 coefficients
                print("Optimized:", rgs_optimized.coef_[k][:5])
        
        # Compare predictions
        y_pred_orig = rgs_original.predict(X_test)
        y_pred_opt = rgs_optimized.predict(X_test)
        pred_diff = np.abs(y_pred_orig - y_pred_opt)
        print(f"\nPrediction differences:")
        print(f"Max difference: {np.max(pred_diff):.6f}")
        print(f"Mean difference: {np.mean(pred_diff):.6f}")
        
        # Store detailed results
        result = {
            'scenario': scenario['name'],
            'n_samples': scenario['n_samples'],
            'n_features': scenario['n_features'],
            'time_original': time_original,
            'time_optimized': time_optimized,
            'speedup': time_original / time_optimized,
            'sets_match': sets_match,
            'coef_diff': np.max(np.abs(rgs_original.coef_ - rgs_optimized.coef_)),
            'pred_diff_max': np.max(pred_diff),
            'pred_diff_mean': np.mean(pred_diff)
        }
        
        # Add MSE comparisons
        for k in range(1, scenario['k_max'] + 1):
            # Training MSE
            y_train_pred_orig = rgs_original.predict(X_train, k=k)
            y_train_pred_opt = rgs_optimized.predict(X_train, k=k)
            result[f'mse_train_orig_k{k}'] = mean_squared_error(y_train, y_train_pred_orig)
            result[f'mse_train_opt_k{k}'] = mean_squared_error(y_train, y_train_pred_opt)
            
            # Test MSE
            y_test_pred_orig = rgs_original.predict(X_test, k=k)
            y_test_pred_opt = rgs_optimized.predict(X_test, k=k)
            result[f'mse_test_orig_k{k}'] = mean_squared_error(y_test, y_test_pred_orig)
            result[f'mse_test_opt_k{k}'] = mean_squared_error(y_test, y_test_pred_opt)
        
        results.append(result)
        
    return pd.DataFrame(results)

# Define simple test scenarios for debugging
debug_scenarios = [
    {
        'name': 'Debug Dataset',
        'n_samples': 2000,
        'n_features': 200,
        'n_informative': 3,
        'k_max': 10,
        'm': 5,
        'n_replications': 1000
    }
]

results_df = run_benchmark(debug_scenarios)

print("\nDetailed Results:")
pd.set_option('display.max_columns', None)
print(results_df.to_string(index=False))


Running benchmark for scenario: Debug Dataset

Testing random state consistency:
Random choices match: True
Generator choices match: True
Original RGS: 11.36 seconds
Optimized RGS: 10.18 seconds

Feature set differences at k=1:
Only in original: [[9], [10], [27], [42], [65], [80], [92], [102], [111], [113], [135], [168]]
Only in optimized: [[1], [2], [6], [8], [11], [13], [14], [15], [16], [17], [19], [20], [22], [23], [24], [25], [26], [30], [32], [33], [34], [35], [36], [38], [39], [40], [44], [45], [49], [50], [51], [53], [54], [55], [56], [57], [59], [60], [61], [62], [63], [64], [67], [68], [74], [76], [77], [79], [84], [85], [87], [88], [89], [90], [91], [94], [96], [97], [98], [99], [104], [108], [114], [115], [116], [117], [119], [122], [123], [124], [125], [126], [128], [130], [131], [134], [137], [138], [140], [141], [142], [144], [145], [150], [153], [154], [155], [158], [159], [160], [162], [163], [164], [165], [166], [167], [169], [171], [172], [173], [175], [179], [180],